In [1]:
from asn1 import Encoder, Decoder, Numbers
from pygost.gost34112012256 import GOST34112012256
from random import randrange


# put signature in asn1 file
def sign_to_asn(a, b, p, q, r, s, x_q, y_q, x_p, y_p, file_name):
    asn1 = Encoder()
    asn1.start()
    asn1.enter(Numbers.Sequence) # heading
    asn1.enter(Numbers.Set) # many keys, 1 involved
    asn1.enter(Numbers.Sequence) # first key
    asn1.write(b'\x80\x06\x07\x00', Numbers.OctetString) # algorithm identifier (GOST signature protocol)
    asn1.enter(Numbers.Sequence) # public key value
    asn1.write(int(x_q), Numbers.Integer) # x-coordinate of point Q 
    asn1.write(int(y_q), Numbers.Integer) # y-coordinate of point Q 
    asn1.leave()
    asn1.enter(Numbers.Sequence) # cryptosystem parameters 
    asn1.enter(Numbers.Sequence) # field parameters 
    asn1.write(p, Numbers.Integer) # prime p
    asn1.leave()
    asn1.enter(Numbers.Sequence) # curve parameters
    asn1.write(a, Numbers.Integer) # coefficient A of the curve equation 
    asn1.write(b, Numbers.Integer) # coefficient B of the curve equation 
    asn1.leave()
    asn1.enter(Numbers.Sequence) # generator of a group of points of a curve
    asn1.write(int(x_p), Numbers.Integer) # x-coordinate of the generator-point P
    asn1.write(int(y_p), Numbers.Integer) # y-coordinate of the generator-point P
    asn1.leave()
    asn1.write(q, Numbers.Integer) # group order q
    asn1.leave()
    asn1.enter(Numbers.Sequence) # message signature
    asn1.write(int(r), Numbers.Integer) # number r
    asn1.write(int(s), Numbers.Integer) # number s
    asn1.leave()
    asn1.leave()
    asn1.leave()
    asn1.enter(Numbers.Sequence) # file parameters (not used)
    asn1.leave()
    asn1.leave()
    with open(file_name + '.sign.asn1', 'wb') as output_file:
        output_file.write(asn1.output())


# достать подпись из asn1 файла
def sign_from_asn(file_name):
    with open(file_name + '.sign.asn1', "rb") as input_file:
        sign_data = input_file.read()
    asn1 = Decoder()
    asn1.start(sign_data)
    asn1.enter() # heading
    asn1.enter() # many keys, 1 involved
    asn1.enter() # first key
    value = asn1.read() # algorithm identifier (GOST signature protocol)
    asn1.enter() # public key value
    x_q = asn1.read()[1] # x-coordinate of point Q
    y_q = asn1.read()[1] # y-coordinate of point Q 
    asn1.leave()
    asn1.enter() # cryptosystem parameters 
    asn1.enter() # field parameters 
    p = asn1.read()[1] # prime p
    asn1.leave() 
    asn1.enter() # curve parameters
    a = asn1.read()[1] # coefficient A of the curve equation 
    b = asn1.read()[1] # coefficient B of the curve equation 
    asn1.leave()
    asn1.enter() # generator of a group of points of a curve
    x_p = asn1.read()[1] # x-coordinate of the generator-point P
    y_p = asn1.read()[1] # y-coordinate of the generator-point P
    asn1.leave()
    q = asn1.read()[1] # group order q
    asn1.leave()
    asn1.enter() # message signature
    r = asn1.read()[1] # number r
    s = asn1.read()[1] # number s
    asn1.leave()
    asn1.leave()
    asn1.leave()
    asn1.enter() # file parameters (not used)
    asn1.leave()
    asn1.leave()
    return a, b, p, q, r, s, x_q, y_q, x_p, y_p


# formation of the signature GOST R 34.10-2018 
def sign_gost34102018(a, b, p, q, d, x_p, y_p, file_name):
    # we read the file with the content that we will sign
    with open(file_name, "rb") as input_file:
        data = input_file.read()
    # set an elliptic curve 
    K = GF(p)
    E = EllipticCurve(K, [a, b])
    P = E(x_p, y_p)
    # 1
    h = GOST34112012256(data).digest()
    # 2
    alpha = int.from_bytes(h, byteorder='big')
    e = mod(alpha, q)
    if e == 0:
        e = 1
    while True:
        # 3
        k = randrange(1, q)
        # 4
        C = k * P
        r = mod(C[0], q)
        if r == 0:
            continue
        # 5
        s = mod(r * d + k * e, q)
        if s == 0:
            continue
        break
    Q = d * P
    sign_to_asn(a, b, p, q, r, s, Q[0], Q[1], P[0], P[1], file_name)
    print('File ' + file_name + ' signed. File with signature: ' + file_name + '.sign.asn1')


# verification of the signature of GOST R 34.10-2018
def check_sign_gost34102018(file_name):
    a, b, p, q, r, s, x_q, y_q, x_p, y_p = sign_from_asn(file_name)
    # we read the file with the content, the signature of which we check 
    with open(file_name, "rb") as input_file:
        data = input_file.read()
    # set an elliptic curve
    K = GF(p)
    E = EllipticCurve(K, [a, b])
    P = E(x_p, y_p)
    Q = E(x_q, y_q)
    # 1
    if not ((0 < r and r < q) and (0 < s and s < q)):
        print('Signature of file ' + file_name + ' is incorrect!')
        return
    # 2
    h = GOST34112012256(data).digest()
    # 3
    alpha = int.from_bytes(h, byteorder='big')
    e = mod(alpha, q)
    if e == 0:
        e = 1
    # 4.
    v = inverse_mod(e, q)
    # 5
    z_1 = int(mod(s * v, q))
    z_2 = int(mod(-r * v, q))
    # 6
    C = z_1 * P + z_2 * Q
    R = mod(C[0], q)
    if R == r:
        print('Signature of file ' + file_name + ' is correct!')
    else:
        print('Signature of file ' + file_name + ' is incorrect!')


print('Signature scheme GOST R 34.10-2018')
# You must specify the path to the signed file 
file_name = 'C:\\test\\image.png'
while True:
    command = int(input('Enter the command:\n1 - sign the file\n2 - check the file signature\n3 - exit\n'))
    if command == 1:
        # The cryptosystem parameters are set statically; in real conditions, a parameter generator is required
        a = -1
        b = 53956679838042162451108292176931772631109916272820066466458395232513766926866
        p = 57896044625259982827082014024491516445703215213774687456785671200359045162371
        r = 28948022312629991413541007012245758222850495633896873081323396140811733708403
        x = 12933162268009944794066590054824622037560826430730236852169234350278155715869
        y = 18786030474197088418858017573086015439132081546303111294023901101650919011383
        q = r
        d = 1732504998467916685624752132319238521322532122193426470314523640303299087831
        sign_gost34102018(a, b, p, q, d, x, y, file_name)
    elif command == 2:
        check_sign_gost34102018(file_name)
    elif command == 3:
        break
    else:
        print('Unknown command!')

Signature scheme GOST R 34.10-2018
Enter the command:
1 - sign the file
2 - check the file signature
3 - exit
3
